In [1]:
!pip install chromadb
!pip install ollama
!pip install tqdm
!pip install psutil

    torch (>=1.7torchvision)
          ~~~~~~^

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
    torch (>=1.7torchvision)
          ~~~~~~^

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
    torch (>=1.7torchvision)
          ~~~~~~^

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
    torch (>=1.7torchvision)
          ~~~~~~^

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [49]:
import ollama

# Initialize Ollama client
client = ollama.Client()

# 1. Function to make a request to mistral-nemo and get a response
def mistral_nemo(prompt):
    response = client.generate(model='mistral-nemo', prompt=prompt)
    return response['response']

# 2. Function to produce embeddings using bge-large model
def get_embeddings(texts):
    embeddings = []
    for text in texts:
        response = client.embeddings(model='bge-large', prompt=text)
        embeddings.append(response['embedding'])
    return embeddings

# Test mistral-nemo
# print("Testing mistral-nemo:")
# prompt = "Explain the concept of machine learning in one sentence."
# response = mistral_nemo(prompt)
# print(f"Prompt: {prompt}")
# print(f"Response: {response}\n")

# Test bge-large embeddings
print("Testing bge-large embeddings:")
test_texts = [
    "The quick brown fox jumps over the lazy dog.",
    "Machine learning is a subset of artificial intelligence."
]
embeddings = get_embeddings(test_texts)
print(f"Number of texts: {len(test_texts)}")
print(f"Number of embeddings: {len(embeddings)}")
print(f"Dimension of each embedding: {len(embeddings[0])}")
print(f"First few values of the first embedding: {embeddings[0]}")

Testing bge-large embeddings:


2024-08-30 16:18:03,342 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"
2024-08-30 16:18:03,363 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"


Number of texts: 2
Number of embeddings: 2
Dimension of each embedding: 1024
First few values of the first embedding: [-1.0442172288894653, 0.006230652332305908, 0.1594274491071701, -0.3180111050605774, -1.172863245010376, -0.4813504219055176, -0.2918023467063904, -0.1900947391986847, 0.9918843507766724, 1.12939453125, 0.7747107148170471, 0.5521367192268372, 0.7885223031044006, -0.03671783208847046, 0.36404451727867126, 0.2962174415588379, -0.5955097079277039, -0.37874162197113037, -0.1500737965106964, 0.13716763257980347, 1.1228944063186646, -0.49782702326774597, -1.0287067890167236, 0.30666887760162354, 0.5918138027191162, 0.0825045257806778, 0.29190346598625183, -0.5599761605262756, 0.909532368183136, 0.7249480485916138, 0.793367862701416, 0.3202283978462219, -0.1045878678560257, -0.7661147117614746, 0.07614733278751373, -0.05575685575604439, -0.13120834529399872, -0.28676357865333557, 0.12181190401315689, -0.4688154458999634, 1.072691559791565, -0.6395083069801331, 0.38099962472915

In [ ]:
text = "abcdefgh"
readme_path = "/Users/dimova01/git/origin_simulator/README.md" 

In [45]:
import chromadb
import ollama
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize Ollama client
ollama_client = ollama.Client()

# Initialize ChromaDB client
client = chromadb.PersistentClient(path="./chroma_db")

# Create a collection
collection = client.get_or_create_collection(name="test_collection")

def process_single_sentence():
    sentence = "This is a test sentence for embedding and storage in ChromaDB."
    logging.info(f"Processing sentence: {sentence}")

    try:
        # Generate embedding
        response = ollama_client.embeddings(model="nomic-embed-text", prompt=sentence)
        embedding = response['embedding']
        logging.info(f"Generated embedding with length: {len(embedding)}")

        # Store in ChromaDB
        collection.add(
            embeddings=[embedding],
            documents=[sentence],
            metadatas=[{"source": "test"}],
            ids=["test_sentence"]
        )
        logging.info("Successfully added sentence to ChromaDB")

        # Retrieve from ChromaDB
        results = collection.get(ids=["test_sentence"])
        logging.info(f"Retrieved from ChromaDB: {results['documents'][0]}")

        # Perform a similarity search
        query_results = collection.query(
            query_embeddings=[embedding],
            n_results=1
        )
        logging.info(f"Query result: {query_results['documents'][0][0]}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

if __name__ == "__main__":
    process_single_sentence()
    logging.info(f"Total documents in collection: {collection.count()}")

2024-08-30 16:07:09,748 - INFO - Processing sentence: This is a test sentence for embedding and storage in ChromaDB.
2024-08-30 16:07:10,306 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"
2024-08-30 16:07:10,307 - INFO - Generated embedding with length: 768
2024-08-30 16:07:10,316 - WARNING - Insert of existing embedding ID: test_sentence
2024-08-30 16:07:10,317 - WARNING - Add of existing embedding ID: test_sentence
2024-08-30 16:07:10,319 - INFO - Successfully added sentence to ChromaDB
2024-08-30 16:07:10,322 - INFO - Retrieved from ChromaDB: This is a test sentence for embedding and storage in ChromaDB.
2024-08-30 16:07:10,327 - INFO - Query result: This is a test sentence for embedding and storage in ChromaDB.
2024-08-30 16:07:10,329 - INFO - Total documents in collection: 1


In [5]:
# import chromadb
# import ollama
# import logging
# import os
# import time
# import psutil

# # Set up logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# # Initialize Ollama client
# ollama_client = ollama.Client()

# # Initialize ChromaDB client
# client = chromadb.PersistentClient(path="./chroma_db")

# # Create or get a collection
# collection = client.get_or_create_collection(name="readme_collection")

# def log_memory_usage():
#     process = psutil.Process(os.getpid())
#     mem = process.memory_info().rss / 1024 / 1024  # in MB
#     logging.info(f"Current memory usage: {mem:.2f} MB")

# def chunk_text(text, chunk_size=500, overlap=50):
#     logging.info(f"Starting to chunk text of length {len(text)} with chunk_size={chunk_size} and overlap={overlap}")
#     chunks = []
#     start = 0
#     while start < len(text):
#         end = start + chunk_size
#         if end > len(text):
#             end = len(text)
#         chunk = text[start:end]
#         chunks.append(chunk)
#         if len(chunks) % 10 == 0:
#             logging.info(f"Created {len(chunks)} chunks so far")
#         start = end - overlap
#         if start >= len(text) - overlap:  # Prevent creating duplicate chunks at the end
#             break
#     logging.info(f"Finished chunking. Created {len(chunks)} chunks")
#     return chunks

# def get_embedding(text):
#     logging.info(f"Getting embedding for text of length {len(text)}")
#     start_time = time.time()
#     try:
#         response = ollama_client.embeddings(model="nomic-embed-text", prompt=text)
#         embedding = response['embedding']
#         end_time = time.time()
#         logging.info(f"Generated embedding of length {len(embedding)} in {end_time - start_time:.2f} seconds")
#         return embedding
#     except Exception as e:
#         logging.error(f"Error generating embedding: {e}")
#         return None

# def process_readme(file_path):
#     logging.info(f"Processing file: {file_path}")
#     log_memory_usage()

#     try:
#         with open(file_path, 'r') as file:
#             content = file.read()
#         logging.info(f"Read file content, length: {len(content)}")
#         log_memory_usage()

#         chunks = chunk_text(content)
#         log_memory_usage()

#         # Clear existing documents in the collection
#         logging.info("Clearing existing documents in the collection")
#         existing_ids = collection.get()["ids"]
#         if existing_ids:
#             collection.delete(ids=existing_ids)
#         log_memory_usage()

#         for i, chunk in enumerate(chunks):
#             logging.info(f"Processing chunk {i+1}/{len(chunks)}")
#             embedding = get_embedding(chunk)
#             if embedding is None:
#                 logging.warning(f"Skipping chunk {i+1} due to embedding error")
#                 continue
            
#             logging.info(f"Adding chunk {i+1} to ChromaDB")
#             collection.add(
#                 embeddings=[embedding],
#                 documents=[chunk],
#                 metadatas=[{"source": file_path, "chunk_id": i}],
#                 ids=[f"chunk_{i}"]
#             )
#             logging.info(f"Successfully added chunk {i+1} to ChromaDB")
#             log_memory_usage()

#         logging.info("Finished processing all chunks")
#         log_memory_usage()

#     except Exception as e:
#         logging.error(f"An error occurred while processing the README: {e}", exc_info=True)


# readme_path = "/Users/dimova01/git/origin_simulator/README.md"
# try:
#     process_readme(readme_path)
#     doc_count = collection.count()
#     logging.info(f"Total documents in collection: {doc_count}")
    
#     if doc_count > 0:
#         # Perform a test query
#         test_query = "What is this project about?"
#         logging.info(f"Performing test query: '{test_query}'")
#         query_embedding = get_embedding(test_query)
#         if query_embedding:
#             results = collection.query(
#                 query_embeddings=[query_embedding],
#                 n_results=1
#             )
#             if results['documents'] and results['documents'][0]:
#                 logging.info(f"Query result: {results['documents'][0][0][:100]}...")  # First 100 characters
#             else:
#                 logging.warning("No results found for the query.")
#         else:
#             logging.error("Failed to generate embedding for test query")
#     else:
#         logging.warning("No documents in the collection. Skipping test query.")
    
# except Exception as e:
#     logging.error(f"An error occurred: {e}", exc_info=True)

# logging.info("Script execution completed")
# log_memory_usage()

In [41]:
# import chromadb
# import ollama
# import logging
# import os
# import time
# from tqdm import tqdm
# from datetime import datetime

# # Set up logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# # Initialize Ollama client
# ollama_client = ollama.Client()

# # Generate a unique collection name with timestamp
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# collection_name = f"repo_collection_{timestamp}"

# # Initialize ChromaDB client with a new path
# chroma_db_path = f"./chroma_db_{timestamp}"
# client = chromadb.PersistentClient(path=chroma_db_path)

# # Create a new collection
# collection = client.create_collection(name=collection_name)

# # Directories to exclude
# EXCLUDED_DIRS = {'deps', '_build'}

# def chunk_text(text, chunk_size=500, overlap=50):
#     chunks = []
#     start = 0
#     while start < len(text):
#         end = start + chunk_size
#         if end > len(text):
#             end = len(text)
#         chunks.append(text[start:end])
#         start = end - overlap
#         if start >= len(text) - overlap:
#             break
#     return chunks

# def get_embedding(text):
#     try:
#         response = ollama_client.embeddings(model="bge-large", prompt=text)
#         return response['embedding']
#     except Exception as e:
#         return None

# def process_file(file_path):
#     try:
#         with open(file_path, 'r', encoding='utf-8') as file:
#             content = file.read()
        
#         chunks = chunk_text(content)
#         for i, chunk in enumerate(chunks):
#             embedding = get_embedding(chunk)
#             if embedding is None:
#                 continue
            
#             collection.add(
#                 embeddings=[embedding],
#                 documents=[chunk],
#                 metadatas=[{"source": file_path, "chunk_id": i}],
#                 ids=[f"{file_path}_chunk_{i}"]
#             )
#         return len(chunks)
#     except Exception:
#         return 0

# def process_repository(repo_path):
#     start_time = time.time()
#     logging.info(f"Starting repository processing: {repo_path}")
#     logging.info(f"Using new collection: {collection_name}")
#     logging.info(f"Storing data in: {chroma_db_path}")

#     total_files = sum([len([f for f in files if f.endswith(('.exs', '.ex', '.md'))])
#                        for r, d, files in os.walk(repo_path)
#                        if not any(excluded in r.split(os.sep) for excluded in EXCLUDED_DIRS)])
    
#     processed_files = 0
#     total_chunks = 0
    
#     with tqdm(total=total_files, desc="Processing files", unit="file") as pbar:
#         for root, dirs, files in os.walk(repo_path):
#             dirs[:] = [d for d in dirs if d not in EXCLUDED_DIRS]
            
#             for file in files:
#                 if file.endswith(('.exs', '.ex', '.md')):
#                     file_path = os.path.join(root, file)
#                     chunks_processed = process_file(file_path)
#                     total_chunks += chunks_processed
#                     processed_files += 1
#                     pbar.update(1)

#     end_time = time.time()
#     processing_time = end_time - start_time

#     return processed_files, total_chunks, processing_time

# if __name__ == "__main__":
#     repo_path = "/Users/dimova01/git/origin_simulator"  # Replace with your repository path
#     try:
#         files_processed, chunks_processed, total_time = process_repository(repo_path)
#         doc_count = collection.count()

#         logging.info("\nProcessing Summary:")
#         logging.info(f"Total files processed: {files_processed}")
#         logging.info(f"Total chunks processed: {chunks_processed}")
#         logging.info(f"Total documents in collection: {doc_count}")
#         logging.info(f"Total processing time: {total_time:.2f} seconds")

#         if doc_count > 0:
#             test_query = "What is this project about?"
#             logging.info(f"\nPerforming test query: '{test_query}'")
#             query_embedding = get_embedding(test_query)
#             if query_embedding:
#                 results = collection.query(
#                     query_embeddings=[query_embedding],
#                     n_results=3
#                 )
#                 for i, doc in enumerate(results['documents'][0]):
#                     logging.info(f"Query result {i+1}: {doc[:200]}...")  # First 200 characters
#             else:
#                 logging.error("Failed to generate embedding for test query")
        
#     except Exception as e:
#         logging.error(f"An error occurred: {e}")
    
#     logging.info("Script execution completed")
#     logging.info(f"Data stored in: {chroma_db_path}")
#     logging.info(f"Collection name: {collection_name}")

In [46]:
import chromadb
import ollama
from typing import List, Dict, Any, Optional

class ChromaDBQuerier:
    def __init__(self, collection_name: str, chroma_db_path: str):
        self.client = chromadb.PersistentClient(path=chroma_db_path)
        self.collection = self.client.get_collection(name=collection_name)
        self.ollama_client = ollama.Client()

    def get_embedding(self, text: str) -> List[float]:
        try:
            response = self.ollama_client.embeddings(model="bge-large", prompt=text)
            return response['embedding']
        except Exception as e:
            print(f"Error generating embedding: {e}")
            return []

    def generate_code(self, query: str) -> str:
        prompt = f"""
        Given the following question about an Elixir project, write Elixir code that might be relevant to answering this question. 
        Focus on creating a code snippet that could be part of the implementation described in the question.
        Do not include any explanations, just the code:

        Question: {query}
        """
        try:
            response = self.ollama_client.generate(model="mistral-nemo", prompt=prompt)
            return response['response'].strip()
        except Exception as e:
            print(f"Error generating code: {e}")
            return ""

    def query(self, query_text: str, n_results: int = 3) -> Dict[str, Any]:
        original_query = query_text
        generated_code = self.generate_code(query_text)
        
        if generated_code:
            query_text = f"{query_text}\n\nRelevant Code:\n{generated_code}"

        query_embedding = self.get_embedding(query_text)
        if not query_embedding:
            return {"error": "Failed to generate embedding"}

        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=n_results,
            include=["documents", "metadatas", "distances"]
        )

        matches = []
        for i in range(len(results['ids'][0])):
            matches.append({
                'document': results['documents'][0][i],
                'metadata': results['metadatas'][0][i],
                'distance': results['distances'][0][i],
            })

        return {
            "original_query": original_query,
            "generated_code": generated_code,
            "matches": matches
        }

    def get_llm_response(self, query_results: Dict[str, Any]) -> str:
        context = "\n\n".join([match['document'] for match in query_results['matches']])
        prompt = f"""
        Based on the following context from an Elixir project's codebase, answer this question:

        Question: {query_results['original_query']}

        Relevant Code from the Project:
        {context}
        """
        try:
            response = self.ollama_client.generate(model="mistral-nemo", prompt=prompt)
            return response['response'].strip()
        except Exception as e:
            print(f"Error generating LLM response: {e}")
            return "Failed to generate response"

    def get_collection_info(self) -> Dict[str, Any]:
        return {
            'name': self.collection.name,
            'count': self.collection.count()
        }

In [47]:
# Replace these with your actual collection name and path
collection_name = "repo_collection_20240830_120758"
chroma_db_path = "./chroma_db_20240830_120758"

querier = ChromaDBQuerier(collection_name, chroma_db_path)

print("Collection Info:", querier.get_collection_info())

query = "How is the HTTP client implemented in this project?"

# Perform the query
query_results = querier.query(query, n_results=3)

# Display intermediate results
print(f"\nOriginal Query: {query_results['original_query']}")
print(f"\nGenerated Code:\n{query_results['generated_code']}")

print("\nRetrieved Matches:")
for i, match in enumerate(query_results['matches'], 1):
    print(f"\nMatch {i}:")
    print(f"Document: {match['document']}...")  # First 200 characters
    print(f"Source: {match['metadata']['source']}")
    print(f"Distance: {match['distance']}")

# Get LLM response
llm_response = querier.get_llm_response(query_results)

print("\nLLM Response:")
print(llm_response)

Collection Info: {'name': 'repo_collection_20240830_120758', 'count': 216}


2024-08-30 16:12:13,513 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2024-08-30 16:12:15,138 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"



Original Query: How is the HTTP client implemented in this project?

Generated Code:
```elixir
defmodule HttpClient do
  def get(url) do
    case :httpc.read(url) do
      {:ok, body} ->
        body
      {:error, %HTTPoison.Error{status_code: 404}} ->
        "Not Found"
      {:error, error} ->
        raise error
    end
  end
end

# Example usage:
# HttpClient.get("https://api.example.com/data")
```

Retrieved Matches:

Match 1:
Document: defmodule Fixtures do
  @http_client Application.compile_env(:origin_simulator, :http_client)

  def body_mock(headers \\ %{}) do
    {:ok, %{body: body}} = @http_client.get("/", headers)
    body
  end

  def http_error_message(status), do: "Error #{status}"

  def origin_payload() do
    %{
      "origin" => "https://www.bbc.co.uk/news",
      "stages" => [%{"at" => 0, "status" => 200, "latency" => 0}],
      "route" => "/news"
    }
  end

  def origin_payload(context), do: {:ok, Map.put(cont...
Source: /Users/dimova01/git/origin_simulator/te

2024-08-30 16:12:44,342 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"



LLM Response:
Based on the provided Elixir project codebase, the HTTP client is implemented using the `HTTPoison` library. Here's how it's done:

1. In the `OriginSimulator.HTTP.Client` module, there are two `get/2` functions that handle the HTTP GET requests with optional headers.

```elixir
defmodule OriginSimulator.HTTP.Client do
  def get(endpoint, headers \\ %{}) do
    # ...
    HTTPoison.get(endpoint, headers |> Map.to_list(), options)
  end

  def get(endpoint, %{"content-encoding" => "gzip"} = headers) do
    new_headers =
      headers
      |> Map.delete("content-encoding")
      |> Map.put("accept-encoding", "gzip")

    get(endpoint, new_headers)
  end
end
```

2. The `get/2` function takes an endpoint and optional headers as arguments. If the `content-encoding` header is present with value `"gzip"`, it's removed and `"accept-encoding"` header is added with value `"gzip"` before making the request.

3. The `HTTPoison.get/3` function is used to send the GET request to the 

In [48]:
# Replace these with your actual collection name and path
collection_name = "repo_collection_20240830_150218"
chroma_db_path = "./chroma_db_20240830_150218"

querier = ChromaDBQuerier(collection_name, chroma_db_path)

print("Collection Info:", querier.get_collection_info())

query = "How can I expand the redirect macro to include only_on: in the same way that it's done in handle macro?"

# Perform the query
query_results = querier.query(query, n_results=3)

# Display intermediate results
print(f"\nOriginal Query: {query_results['original_query']}")
print(f"\nGenerated Code:\n{query_results['generated_code']}")

print("\nRetrieved Matches:")
for i, match in enumerate(query_results['matches'], 1):
    print(f"\nMatch {i}:")
    print(f"Document: {match['document']}...")
    print(f"Source: {match['metadata']['source']}")
    print(f"Distance: {match['distance']}")

# Get LLM response
llm_response = querier.get_llm_response(query_results)

print("\nLLM Response:")
print(llm_response)

2024-08-30 16:13:54,792 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Collection Info: {'name': 'repo_collection_20240830_150218', 'count': 11077}


2024-08-30 16:14:03,746 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2024-08-30 16:14:04,778 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"



Original Query: How can I expand the redirect macro to include only_on: in the same way that it's done in handle macro?

Generated Code:
```elixir
defmodule MyApp.Web, :macro do
  def redirect(only_on: path) do
    {:ok, _conn, %{assigns: assigns}} =
      Phoenix.Controller.redirect(to: path, assigns: assigns)
    Macro.escape_redirect_path(path)
    |> Macro.expand_forwards(only_on)
    |> {:_redirect, assigns}
  end

  defp escape_redirect_path(path) do
    String.replace(path, ~r/"/, ~r/\\\/)
  end

  defp expand_forwards(path, only_on) do
    if path == only_on, do: path,
      else: "#{path} #{only_on}"
  end
end
```

Retrieved Matches:

Match 1:
Document: , Map.put_new(args, :only_on, nil)}
        end)
      end
    end
  end

  defp defs() do
    quote unquote: false do
      var!(add_redirect, BelfrageWeb.RouteMaster) = fn from, location, status, ttl ->
        matcher = BelfrageWeb.ReWrite.prepare(location) |> Macro.escape()

        redirect_statuses = Application.compile_

2024-08-30 16:14:30,835 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"



LLM Response:
To expand the `redirect` macro to include the `:only_on` option like it's done in the `handle` macro, you'll need to modify the `redirect/4` macro and add a new `add_redirect_for_env_with_block/6` function similar to `add_route_for_env_with_block/5`. Here's how you can achieve this:

1. Add a new function `add_redirect_for_env_with_block/6`:

```elixir
defmodule BelfrageWeb do
  # ... other functions ...

  defp add_redirect_for_env_with_block(matcher, from, location, status, ttl, env) do
    # Implement the logic to add a redirect with environment conditions here.
    # This function should be similar to `add_route_for_env_with_block/5`.
    IO.inspect {:ok, matcher}
  end

end
```

2. Modify the `redirect/4` macro:

```elixir
defmacro redirect(matcher, [from: from, to: location, status: status, ttl: ttl | env_options] = args) do
  quote bind_quoted: [
            matcher: matcher,
            from: from,
            location: location,
            status: status,
     